In [6]:
import numpy as np
import torch
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
import os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torchvision.models as models
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.nn.functional as F

<H2> Data prep

In [7]:
#transform to 224, 224 since VGG expect that size
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])

# Define paths to your train and validation data
train_path = "train"
val_path = "val"

#define train and valdataset again but with transoformers this time
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

#8 batch size to reduce training time
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#20% of training since otherwise it would have taken a day to train
subset_size = int(0.2 * len(train_dataset))

# Randomly select a subset of indices from the training dataset
subset_indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# Create a subset of the training dataset using the selected indices
train_subset = Subset(train_dataset, subset_indices)

# Use DataLoader to load the subset of data in batches
train_loader_subset = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

# Print dataset details
print("Number of classes:", len(train_dataset.classes))
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))
print("Number of training subset images:", len(train_subset))
print("Number of validation images:", len(val_dataset))

Number of classes: 2
Classes: ['MEL', 'NV']
Number of training images: 6426
Number of training subset images: 1285
Number of validation images: 1252


<h2> Some baseline model

In [4]:
class SimpleCNN(nn.Module):
    
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Training)", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=correct / total)

    # Print training statistics
    epoch_loss = running_loss / len(train_subset)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.2%}")

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)", unit="batch"):
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item() * val_inputs.size(0)
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    # Print validation statistics
    val_epoch_loss = val_loss / len(val_dataset)
    val_epoch_acc = val_correct / val_total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2%}")


Epoch 1/5 (Training): 100%|██████████| 804/804 [00:46<00:00, 17.47batch/s, accuracy=0.744, loss=0.505]


Epoch [1/5], Training Loss: 2.5237, Training Accuracy: 74.43%


Epoch 1/5 (Validation): 100%|██████████| 157/157 [00:03<00:00, 45.56batch/s]


Epoch [1/5], Validation Loss: 0.4728, Validation Accuracy: 76.68%


Epoch 2/5 (Training): 100%|██████████| 804/804 [00:32<00:00, 24.58batch/s, accuracy=0.778, loss=0.442]


...

Epoch [4/5], Validation Loss: 0.4542, Validation Accuracy: 77.08%


Epoch 5/5 (Training): 100%|██████████| 804/804 [00:32<00:00, 24.62batch/s, accuracy=0.786, loss=0.442]


Epoch [5/5], Training Loss: 2.2127, Training Accuracy: 78.56%


Epoch 5/5 (Validation): 100%|██████████| 157/157 [00:02<00:00, 55.58batch/s]

Epoch [5/5], Validation Loss: 0.4303, Validation Accuracy: 77.00%


## Baseline model with batch normalization

In [21]:
class SimpleCNNwithBN(nn.Module):
    
    def __init__(self, num_classes):
        super(SimpleCNNwithBN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)  
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)  
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)  
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [22]:
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
num_classes = len(train_dataset.classes)
model = SimpleCNNwithBN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Training)", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            t.set_postfix(loss=running_loss / total, accuracy=correct / total)

    # Print training statistics
    epoch_loss = running_loss / len(train_subset)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.2%}")

    # Validation phase
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    with torch.no_grad():
        for val_inputs, val_labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} (Validation)", unit="batch"):
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item() * val_inputs.size(0)
            _, val_predicted = torch.max(val_outputs, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    # Print validation statistics
    val_epoch_loss = val_loss / len(val_dataset)
    val_epoch_acc = val_correct / val_total
    print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_epoch_loss:.4f}, Validation Accuracy: {val_epoch_acc:.2%}")


Epoch 1/5 (Training): 100%|██████████| 804/804 [00:38<00:00, 20.68batch/s, accuracy=0.726, loss=1.3] 


Epoch [1/5], Training Loss: 6.5191, Training Accuracy: 72.61%


Epoch 1/5 (Validation): 100%|██████████| 157/157 [00:03<00:00, 43.27batch/s]


Epoch [1/5], Validation Loss: 0.4328, Validation Accuracy: 77.96%


Epoch 2/5 (Training): 100%|██████████| 804/804 [00:36<00:00, 21.80batch/s, accuracy=0.755, loss=0.492]


...

Epoch [4/5], Validation Loss: 0.4470, Validation Accuracy: 71.41%


Epoch 5/5 (Training): 100%|██████████| 804/804 [00:35<00:00, 22.72batch/s, accuracy=0.719, loss=0.53] 


Epoch [5/5], Training Loss: 2.6504, Training Accuracy: 71.93%


Epoch 5/5 (Validation): 100%|██████████| 157/157 [00:02<00:00, 52.64batch/s]

Epoch [5/5], Validation Loss: 0.4829, Validation Accuracy: 78.67%


<h2> VGG16 Model

In [14]:
num_cpu_cores = os.cpu_count()
print(num_cpu_cores)

12


In [15]:
random_transforms = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.15, saturation=0.1, hue=0.1)
], p=0.2)  # Apply each transformation with 20% probability

# Use the provided vggtransforms for preprocessing
transform = transforms.Compose([
    random_transforms,               # Apply random transformations for data augmentation
    transforms.Resize((224, 224)),   # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  #preprocess images according to
    #imagenet numbers
])

#define train and valdataset again but with complex transformers
#need to redefine everything so the new transformers are applied correctly
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_cpu_cores)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_cpu_cores)

subset_size = int(0.2 * len(train_dataset))

subset_indices = np.random.choice(len(train_dataset), subset_size, replace=False)

train_subset = Subset(train_dataset, subset_indices)

train_loader_subset = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_cpu_cores)

In [16]:

#load in weights from the model
weights_id = torchvision.models.VGG16_Weights.IMAGENET1K_V1

vggmodel = models.vgg16(weights=weights_id)
vggmodel.eval()  # Set the model to evaluation mode

# Freeze all layers except the last classifier layer
for param in vggmodel.parameters():
    param.requires_grad = False

# Unfreeze the last classifier layer
for param in vggmodel.classifier[6].parameters():
    param.requires_grad = True

# Modify the classifier layer
num_features = vggmodel.classifier[6].in_features
vggmodel.classifier[6] = nn.Linear(num_features, len(train_dataset.classes))

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg_model = vggmodel.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vggmodel.parameters(), lr=0.001)

In [17]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    vgg_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # Initialize tqdm progress bar
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = vgg_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total_train, accuracy=correct_train / total_train)

    # Calculate training accuracy
    train_accuracy = correct_train / total_train

    # Validation phase
    vgg_model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vgg_model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = correct_val / total_val

    # Print epoch statistics
    epoch_loss = running_loss / len(train_subset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}, Val Accuracy: {val_accuracy:.2%}")

Epoch 1/5: 100%|██████████| 804/804 [01:48<00:00,  7.42batch/s, accuracy=0.796, loss=0.454]


Epoch [1/5], Loss: 2.2712, Train Accuracy: 79.65%, Val Accuracy: 82.59%


Epoch 2/5: 100%|██████████| 804/804 [01:47<00:00,  7.47batch/s, accuracy=0.801, loss=0.466]


Epoch [2/5], Loss: 2.3298, Train Accuracy: 80.14%, Val Accuracy: 80.27%


Epoch 3/5: 100%|██████████| 804/804 [01:47<00:00,  7.46batch/s, accuracy=0.814, loss=0.442]


Epoch [3/5], Loss: 2.2089, Train Accuracy: 81.37%, Val Accuracy: 81.39%


Epoch 4/5: 100%|██████████| 804/804 [01:47<00:00,  7.47batch/s, accuracy=0.811, loss=0.475]


Epoch [4/5], Loss: 2.3755, Train Accuracy: 81.09%, Val Accuracy: 82.83%


Epoch 5/5: 100%|██████████| 804/804 [01:47<00:00,  7.45batch/s, accuracy=0.805, loss=0.487]


Epoch [5/5], Loss: 2.4351, Train Accuracy: 80.55%, Val Accuracy: 83.87%


Epoch [5/5], Loss: 2.3846, Train Accuracy: 80.91%, Val Accuracy: 80.43% all transformations

In [25]:
# Set the model to evaluation mode
vggmodel.eval()

# Variables to keep track of accuracy and loss
val_loss = 0.0
val_correct = 0
val_total = 0

# Use tqdm for progress bar
with tqdm(val_loader, desc="Validation", unit="batch") as t:
    # Disable gradient computation for validation
    with torch.no_grad():
        # Iterate over the validation set
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = vggmodel(inputs)
            loss = criterion(outputs, labels)

            # Compute loss
            val_loss += loss.item() * inputs.size(0)

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=val_loss / val_total, accuracy=val_correct / val_total)

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(val_dataset)
val_accuracy = val_correct / val_total

# Print validation results
print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")


Validation: 100%|██████████| 157/157 [00:21<00:00,  7.29batch/s, accuracy=0.842, loss=0.358]

Validation Loss: 0.3579, Validation Accuracy: 84.19%


<H2> ResNet Try

In [19]:
# Define settings
use_data_augmentation = True
use_batch_norm = True
use_weight_decay = True
use_subset_training = True
batch_size = 8
lr = 0.0001
num_epochs = 5

In [20]:
# Define transforms
random_transforms = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.15, saturation=0.1, hue=0.1)
], p=0.2) if use_data_augmentation else None

transform = transforms.Compose([
    random_transforms,
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_cpu_cores)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_cpu_cores)

# Subset training data if enabled
if use_subset_training:
    subset_size = int(0.2 * len(train_dataset))
    subset_indices = np.random.choice(len(train_dataset), subset_size, replace=False)
    train_subset = Subset(train_dataset, subset_indices)
    train_loader_subset = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_cpu_cores)
else:
    train_loader_subset = train_loader

In [22]:
# Load pre-trained ResNet model
resnet_model = models.resnet18(weights=True)

# Freeze the parameters of the model
if not use_batch_norm:
   # Unfreeze last layer
    for param in resnet_model.fc.parameters():
        param.requires_grad = True

# Add batch normalization if enabled
if use_batch_norm:
    for param in resnet_model.fc.parameters():
        param.requires_grad = True
    for module in resnet_model.modules():
        if isinstance(module, nn.Conv2d):
            module.add_module('batch_norm', nn.BatchNorm2d(module.out_channels))

c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
# Modify the classifier layer
num_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_features, len(train_dataset.classes))

# Move the model to the appropriate device
resnet_model = resnet_model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_params = {'lr': lr}
if use_weight_decay:
    optimizer_params['weight_decay'] = 0.0001
optimizer = optim.Adam(resnet_model.parameters(), **optimizer_params)

In [24]:
from sklearn.metrics import confusion_matrix
import numpy as np
true_labels = []
predicted_labels = []
# Training loop
for epoch in range(num_epochs):
    resnet_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Initialize tqdm progress bar
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = resnet_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total_train, accuracy=correct_train / total_train)

    # Calculate training accuracy
    train_accuracy = correct_train / total_train

    # Validation phase
    resnet_model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet_model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = correct_val / total_val

    # Print epoch statistics
    epoch_loss = running_loss / len(train_subset) if use_subset_training else running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}, Val Accuracy: {val_accuracy:.2%}")


with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet_model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        # Convert tensors to numpy arrays and store in lists
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()

# Calculate validation accuracy
val_accuracy = correct_val / total_val

# Print validation accuracy
print(f"Validation Accuracy: {val_accuracy:.2%}")

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


Epoch 1/5: 100%|██████████| 804/804 [01:18<00:00, 10.24batch/s, accuracy=0.823, loss=0.396]


Epoch [1/5], Loss: 1.9795, Train Accuracy: 82.32%, Val Accuracy: 85.14%


Epoch 2/5: 100%|██████████| 804/804 [01:17<00:00, 10.43batch/s, accuracy=0.858, loss=0.33] 


Epoch [2/5], Loss: 1.6496, Train Accuracy: 85.84%, Val Accuracy: 86.50%


Epoch 3/5: 100%|██████████| 804/804 [01:17<00:00, 10.34batch/s, accuracy=0.87, loss=0.295] 


Epoch [3/5], Loss: 1.4766, Train Accuracy: 87.02%, Val Accuracy: 85.78%


Epoch 4/5: 100%|██████████| 804/804 [01:14<00:00, 10.72batch/s, accuracy=0.898, loss=0.248]


Epoch [4/5], Loss: 1.2416, Train Accuracy: 89.82%, Val Accuracy: 87.54%


Epoch 5/5: 100%|██████████| 804/804 [01:14<00:00, 10.74batch/s, accuracy=0.911, loss=0.213]


Epoch [5/5], Loss: 1.0647, Train Accuracy: 91.13%, Val Accuracy: 88.58%
Validation Accuracy: 88.62%
Confusion Matrix:
[[557  69]
 [ 73 553]]


In [ ]:
# Create a directory to save misclassified images
output_dir = 'misclassified_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Lists to store misclassified image paths, labels, and predictions
misclassified_image_paths = []
misclassified_labels = []
misclassified_predictions = []

# Validation phase
resnet_model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = resnet_model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()
        
        # Find misclassified images
        misclassified_mask = predicted != labels
        misclassified_images = inputs[misclassified_mask]
        misclassified_labels.extend(labels[misclassified_mask].tolist())
        misclassified_predictions.extend(predicted[misclassified_mask].tolist())
        
        # Save misclassified images
        for i, image in enumerate(misclassified_images):
            image_path = os.path.join(output_dir, f'misclassified_{len(misclassified_image_paths) + 1}.png')
            misclassified_image_paths.append(image_path)
            
            # Normalize image pixel values to the range [0, 1]
            image = image.cpu().numpy().transpose((1, 2, 0))
            image = (image - image.min()) / (image.max() - image.min())
            
            plt.imsave(image_path, image)

## Resnet without data augmentation

In [15]:
# Define settings
use_batch_norm = True
use_weight_decay = True
use_subset_training = True
batch_size = 8
lr = 0.0001
num_epochs = 5

In [17]:
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# Load pre-trained ResNet model
resnet_model = models.resnet18(weights=True)

# Freeze the parameters of the model
if not use_batch_norm:
   # Unfreeze last layer
    for param in resnet_model.fc.parameters():
        param.requires_grad = True

# Add batch normalization if enabled
if use_batch_norm:
    for param in resnet_model.fc.parameters():
        param.requires_grad = True
    for module in resnet_model.modules():
        if isinstance(module, nn.Conv2d):
            module.add_module('batch_norm', nn.BatchNorm2d(module.out_channels))

c:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Modify the classifier layer
num_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_features, len(train_dataset.classes))

# Move the model to the appropriate device
resnet_model = resnet_model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_params = {'lr': lr}
if use_weight_decay:
    optimizer_params['weight_decay'] = 0.0001
optimizer = optim.Adam(resnet_model.parameters(), **optimizer_params)

In [20]:
true_labels = []
predicted_labels = []
# Training loop
for epoch in range(num_epochs):
    resnet_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Initialize tqdm progress bar
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = resnet_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total_train, accuracy=correct_train / total_train)

    # Calculate training accuracy
    train_accuracy = correct_train / total_train

    # Validation phase
    resnet_model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet_model(inputs)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    # Calculate validation accuracy
    val_accuracy = correct_val / total_val

    # Print epoch statistics
    epoch_loss = running_loss / len(train_subset) if use_subset_training else running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Accuracy: {train_accuracy:.2%}, Val Accuracy: {val_accuracy:.2%}")


Epoch 1/5: 100%|██████████| 804/804 [01:02<00:00, 12.89batch/s, accuracy=0.819, loss=0.395]


Epoch [1/5], Loss: 1.9751, Train Accuracy: 81.95%, Val Accuracy: 85.86%


Epoch 2/5: 100%|██████████| 804/804 [01:01<00:00, 13.15batch/s, accuracy=0.866, loss=0.308]


Epoch [2/5], Loss: 1.5396, Train Accuracy: 86.57%, Val Accuracy: 85.06%


Epoch 3/5: 100%|██████████| 804/804 [01:01<00:00, 12.98batch/s, accuracy=0.896, loss=0.255]


Epoch [3/5], Loss: 1.2748, Train Accuracy: 89.59%, Val Accuracy: 86.98%


Epoch 4/5: 100%|██████████| 804/804 [01:01<00:00, 13.10batch/s, accuracy=0.914, loss=0.21] 


Epoch [4/5], Loss: 1.0521, Train Accuracy: 91.36%, Val Accuracy: 87.14%


Epoch 5/5: 100%|██████████| 804/804 [01:01<00:00, 12.97batch/s, accuracy=0.942, loss=0.157]


Epoch [5/5], Loss: 0.7868, Train Accuracy: 94.18%, Val Accuracy: 85.78%
